In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import obspy
import numpy as np
import pandas as pd
import glob 
import scipy

from pylops.utils                      import dottest
from pylops.utils.wavelets             import *

In [2]:
from pyfrac.modelling.kirchhoff import Kirchhoff

# VISUALISATION
from pyfrac.visualisation.traceviz import traceimage
from pyfrac.visualisation.eventimages import locimage3d

# DATA

# RECEIVER GEOMETRY

In [3]:
rec_file = '/home/birniece/Documents/data/ToC2ME/Receivers/ToC2ME_Demo_recloc.txt'
recDF = pd.read_csv(rec_file)


# determine lateral model extents from receiver geometry
xmin = min(recDF['NAD83_X_m'])
xmax = max(recDF['NAD83_X_m'])
print('X', xmin, xmax)

ymin = min(recDF['NAD83_Y_m'])
ymax = max(recDF['NAD83_Y_m'])
print('Y', ymin, ymax)

zmin = min(recDF['z_m'])
zmax = max(recDF['z_m'])
print('Z', zmin, zmax)


datum = -980

X 1234.56709999999 7849.87270000001
Y 2072.62100000028 9502.92899999954
Z 816.2254735 971.0501945


In [4]:
rec_x = recDF['NAD83_X_m'] - xmin
rec_y = recDF['NAD83_Y_m'] - ymin
rec_z = abs(recDF['z_m'] + datum)

recs = np.vstack((rec_x, rec_y, rec_z))
recs = np.round(recs, decimals=-1)
nr = recs.shape[1]

# VELOCITY MODEL

In [5]:
vmod_file = '/home/birniece/Documents/data/ToC2ME/VelocityModels/ToC2MEVelModel.mat'
mod_zprofile = scipy.io.loadmat(vmod_file)

z = mod_zprofile['z'] + datum
vp_1D = mod_zprofile['vp']
# vs_1D = mod_zprofile['vs']
# rho_1D = mod_zprofile['rho']

In [6]:
d_xyz = 10  # meter sampling
mod_xmin = np.round(xmin,decimals=-1) 
mod_xmax = np.round(xmax,decimals=-1) 

mod_ymin = np.round(ymin,decimals=-1)
mod_ymax = np.round(ymax,decimals=-1)

mod_zmin = datum
mod_zmax = 3500   # based on catalogue 

mod_x = np.arange(mod_xmin, mod_xmax, d_xyz)
mod_y = np.arange(mod_ymin, mod_ymax, d_xyz)
mod_z = np.arange(mod_zmin, mod_zmax, d_xyz)

nx_mod = len(mod_x)
ny_mod = len(mod_y)
nz_mod = len(mod_z)

In [7]:
f_vp = scipy.interpolate.interp1d(z.flatten(), vp_1D.flatten(), kind='previous')
vp_log = f_vp(mod_z)

In [8]:
vp_pancake = np.expand_dims(vp_log, axis=[1,2])
vp_pancake = vp_pancake.repeat(nx_mod,axis=2).repeat(ny_mod,axis=1).transpose([2,1,0])

In [9]:
vp_pancake.shape

(662, 743, 448)

# BUILD OPERATOR

In [10]:
nx, ny, nz = vp_pancake.shape
dx, dy, dz = d_xyz, d_xyz, d_xyz
x, y, z = np.arange(nx)*dx, np.arange(ny)*dy, np.arange(nz)*dz

In [11]:
nt = 501
dt = 0.002
t = np.arange(nt)*dt
wav, wavt, wavc = ricker(t[:41], f0=20)

In [12]:
recs.shape

(3, 69)

In [ ]:
# INITIALISE OPERATOR
Op = Kirchhoff(z=z, 
               x=x, 
               y=y, 
               t=t, 
               srcs=recs[:, :1], 
               recs=recs, 
               vel=vp_pancake, 
               wav=wav, 
               wavcenter=wavc, 
               mode='eikonal', 
               engine='numba')

# check operator with dottest
_ = dottest(Op, verb=True)

/home/birniece/Documents/Projects/MicroseismicModelling/MicroseismicProcessing/pyfrac/modelling/kirchhoff.py:255: FutureWarning: A new implementation of Kirchhoff is provided in v2.1.0. This currently affects only the inner working of the operator, end-users can continue using the operator in the same way. Nevertheless, it is now recommended to providethe variables trav (and amp) as a tuples containing the traveltime (and amplitude) tables for sources and receivers separately. This behaviour will eventually become default in version v3.0.0.
  warnings.warn(


[219   4   6]
